In [1]:
import tensorflow as tf

tf.keras.layers.LSTMCell()

In [3]:
v=tf.Variable(0.0)
(v+1).numpy()

1.0

In [7]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [8]:
v.assign_add(1)
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [10]:
w=tf.Variable([[1.0]])
with tf.GradientTape() as t: #自动跟踪变量的梯度计算
    loss = w*w


In [11]:
grad=t.gradient(loss,w)
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [15]:
d=tf.constant([[3.0]])
with tf.GradientTape() as t: #自动跟踪变量的梯度计算
    t.watch(d) #常量要用watch
    dloss = d*d

In [16]:
dloss_d = t.gradient(dloss,d)
dloss_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

In [21]:
d=tf.constant([[3.0]])
with tf.GradientTape(persistent=True) as t: #持久
    t.watch(d) #常量要用watch
    y = d*d
    z = y*y

dy_d = t.gradient(y,d)
dy_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

In [22]:
dz_d = t.gradient(z,d)
dz_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[108.]], dtype=float32)>

In [23]:
(train_image,train_labels),_=tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 166s 14us/step

In [26]:
train_image=tf.expand_dims(train_image,-1)

In [27]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [28]:
train_image = tf.cast(train_image/255,tf.float32)

In [29]:
train_labels = tf.cast(train_labels,tf.int64)

In [30]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [31]:
dataset = dataset.shuffle(10000).batch(32).repeat(1)
dataset

<RepeatDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [33]:
optimizer=tf.keras.optimizers.Adam()

In [34]:
loss_fun=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

标签为顺序编码

sparse_categorical_crossentropy() 需要指定y_true和y_pred

SparseCategoricalCrossentropy() 直接调用，不需要指定y_true和y_pred

标签为独热编码

categorical_crossentropy()


In [35]:
feature,labels = next(iter(dataset))
feature.shape

TensorShape([32, 28, 28, 1])

In [36]:
predictions = model(feature)
predictions.shape

TensorShape([32, 10])

In [37]:
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int64)>

In [38]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([4, 2, 0, 9, 1, 5, 7, 9, 8, 0, 0, 3, 9, 3, 6, 1, 1, 9, 1, 7, 2, 9,
       1, 8, 9, 6, 6, 5, 1, 7, 2, 6], dtype=int64)>

In [39]:
def loss(model,x,y):
    y_ = model(x)
    return loss_fun(y,y_)

In [40]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        loss_step = loss(model,images,labels)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))

In [41]:
def train():
    for epoch in range(10):
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print("Epoch{} is finshed".format(epoch))



In [42]:
train()


Epoch0 is finshed
Epoch1 is finshed
Epoch2 is finshed
Epoch3 is finshed
Epoch4 is finshed
Epoch5 is finshed
Epoch6 is finshed
Epoch7 is finshed
Epoch8 is finshed
Epoch9 is finshed
